<a href="https://colab.research.google.com/github/PraneethaRajupalepu/Bank-DataScience-Project/blob/main/bank_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Loading the dataset**

In [1]:
#access the data files

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score


In [3]:
bank = pd.read_csv('/content/drive/MyDrive/DB2/bank-full.csv', delimiter=";")
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# **Cleaning the data**

In [4]:
bank['default'] = bank['default'].map( {'yes':1 ,'no':0}) 
bank['housing'] = bank['housing'].map( {'yes':1 ,'no':0}) 
bank['loan'] = bank['loan'].map(  {'yes':1 ,'no':0}) 
bank['y'] = bank['y'].map( {'yes':1 ,'no':0}) 

In [5]:
bank.drop(bank[bank['job'] == 'unknown' ].index , inplace=True)
bank.drop(bank[bank['marital'] == 'unknown' ].index , inplace=True)
bank.drop(bank[bank['education'] == 'unknown' ].index , inplace=True)
bank.drop(bank[bank['default'] == 'unknown' ].index , inplace=True)
bank.drop(bank[bank['housing'] == 'unknown' ].index , inplace=True)
bank.drop(bank[bank['loan'] == 'unknown' ].index , inplace=True)

## **Encoding the Data**

In [6]:
label_encoder = LabelEncoder()

In [7]:
integer_encoded = label_encoder.fit_transform(array(list(bank.month)))
dummy_list  = list(integer_encoded)
bank['month'] = dummy_list

In [8]:
integer_encoded = label_encoder.fit_transform(array(list(bank.job)))
dummy_list  = list(integer_encoded)
bank['job'] = dummy_list

In [9]:
integer_encoded = label_encoder.fit_transform(array(list(bank.marital)))
dummy_list  = list(integer_encoded)
bank['marital'] = dummy_list

In [10]:
integer_encoded = label_encoder.fit_transform(array(list(bank.education)))
dummy_list  = list(integer_encoded)
bank['education'] = dummy_list

In [11]:
integer_encoded = label_encoder.fit_transform(array(list(bank.contact)))
dummy_list  = list(integer_encoded)
bank['contact'] = dummy_list

In [12]:
integer_encoded = label_encoder.fit_transform(array(list(bank.poutcome)))
dummy_list  = list(integer_encoded)
bank['poutcome'] = dummy_list

## **Splitting the Dataset**

In [13]:
bank = bank.drop(['day', 'month', 'contact', 'poutcome'], axis = 1)
bank

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,y
0,58,4,1,2,0,2143,1,0,261,1,-1,0,0
1,44,9,2,1,0,29,1,0,151,1,-1,0,0
2,33,2,1,1,0,2,1,1,76,1,-1,0,0
5,35,4,1,2,0,231,1,0,139,1,-1,0,0
6,28,4,2,2,0,447,1,1,217,1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,977,3,-1,0,1
45207,71,5,0,0,0,1729,0,0,456,2,-1,0,1
45208,72,5,1,1,0,5715,0,0,1127,5,184,3,1
45209,57,1,1,1,0,668,0,0,508,4,-1,0,0


In [14]:
y=bank["y"]


In [15]:
X=bank[bank.columns[0:12]]
X

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous
0,58,4,1,2,0,2143,1,0,261,1,-1,0
1,44,9,2,1,0,29,1,0,151,1,-1,0
2,33,2,1,1,0,2,1,1,76,1,-1,0
5,35,4,1,2,0,231,1,0,139,1,-1,0
6,28,4,2,2,0,447,1,1,217,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,977,3,-1,0
45207,71,5,0,0,0,1729,0,0,456,2,-1,0
45208,72,5,1,1,0,5715,0,0,1127,5,184,3
45209,57,1,1,1,0,668,0,0,508,4,-1,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## **Model training and Evaluation**

**1-Logistic regression**

In [17]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [18]:
predictions = logmodel.predict(X_test)
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.91      0.98      0.94      7678\n           1       0.57      0.19      0.29       961\n\n    accuracy                           0.89      8639\n   macro avg       0.74      0.59      0.61      8639\nweighted avg       0.87      0.89      0.87      8639\n'

In [19]:
confusion_matrix(y_test,predictions)


array([[7539,  139],
       [ 778,  183]])

In [20]:
accuracy_score(y_test,predictions)

0.8938534552610256

In [21]:
f1_score(y_test,predictions)

0.2852689010132502

## **2- Naive Bayes**

In [22]:
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
NBmodel = GaussianNB()

# Train the model using the training sets
NBmodel.fit(X_train,y_train)


GaussianNB()

In [23]:

#Predict Output
predictions = NBmodel.predict(X_test)
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.92      0.94      0.93      7678\n           1       0.43      0.38      0.41       961\n\n    accuracy                           0.87      8639\n   macro avg       0.68      0.66      0.67      8639\nweighted avg       0.87      0.87      0.87      8639\n'

In [24]:
confusion_matrix(y_test,predictions)


array([[7190,  488],
       [ 593,  368]])

In [25]:
accuracy_score(y_test,predictions)

0.8748697765945133

In [26]:
f1_score(y_test,predictions)

0.40506329113924044

 **3- Ensumble Modles (logistic regression, xgboost, and random forest) using max voting**

In [27]:

# importing utility modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
 
# importing machine learning models for prediction
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
 
# importing voting classifier
from sklearn.ensemble import VotingClassifier
 
 
# initializing all the model objects with default parameters
model_1 = LogisticRegression()
model_2 = XGBClassifier()
model_3 = RandomForestClassifier()
 
# Making the final model using voting classifier
final_model = VotingClassifier(
    estimators=[('lr', model_1), ('xgb', model_2), ('rf', model_3)], voting='hard')
 
# training all the model on the train dataset
final_model.fit(X_train, y_train)
 
# predicting the output on the test dataset
pred_final = final_model.predict(X_test)
 
# printing log loss between actual and predicted value
print(log_loss(y_test, pred_final))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


3.426308796707412


In [28]:
#Predict Output
predictions = final_model.predict(X_test)
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.92      0.98      0.95      7678\n           1       0.61      0.30      0.41       961\n\n    accuracy                           0.90      8639\n   macro avg       0.76      0.64      0.68      8639\nweighted avg       0.88      0.90      0.89      8639\n'

In [29]:
confusion_matrix(y_test,predictions)


array([[7490,  188],
       [ 669,  292]])

In [30]:
accuracy_score(y_test,predictions)

0.900798703553652

In [31]:
f1_score(y_test,predictions)

0.4052741151977793

 **4- Ensumble Models (Bagging)**

In [32]:
# importing utility modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
 
# importing machine learning models for prediction
import xgboost as xgb
 
# importing bagging module
from sklearn.ensemble import BaggingRegressor
 
 
# initializing the bagging model using XGboost as base model with default parameters
Bagmodel = BaggingRegressor(base_estimator=xgb.XGBRegressor())
 
# training model
Bagmodel.fit(X_train, y_train)
 
# predicting the output on the test dataset
pred = Bagmodel.predict(X_test)
 
# printing the root mean squared error between real value and predicted value
print(mean_squared_error(y_test, pred_final))

[15:51:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:51:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [33]:
classification_report(y_test,pred_final)

'              precision    recall  f1-score   support\n\n           0       0.92      0.98      0.95      7678\n           1       0.61      0.30      0.41       961\n\n    accuracy                           0.90      8639\n   macro avg       0.76      0.64      0.68      8639\nweighted avg       0.88      0.90      0.89      8639\n'

In [34]:
confusion_matrix(y_test,pred_final)


array([[7490,  188],
       [ 669,  292]])

In [35]:
accuracy_score(y_test,pred_final)

0.900798703553652

In [36]:
f1_score(y_test,predictions)

0.4052741151977793

**`5- Stacking classifier (RF, KNN - DT, RF)`**

In [37]:

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = load_iris(return_X_y=True)

# Create Learners per layer
layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))             
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

clf.fit(X_train, y_train)

StackingClassifier(estimators=[('rf_1',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('knn_1', KNeighborsClassifier())],
                   final_estimator=StackingClassifier(estimators=[('dt_2',
                                                                   DecisionTreeClassifier()),
                                                                  ('rf_2',
                                                                   RandomForestClassifier(n_estimators=50,
                                                                                          random_state=42))],
                                                      final_estimator=LogisticRegression()))

In [38]:
predictions = clf.predict(X_test)
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.91      0.97      0.94      7678\n           1       0.54      0.28      0.37       961\n\n    accuracy                           0.89      8639\n   macro avg       0.73      0.62      0.65      8639\nweighted avg       0.87      0.89      0.88      8639\n'

In [39]:
confusion_matrix(y_test,predictions)


array([[7446,  232],
       [ 692,  269]])

In [40]:
accuracy_score(y_test,pred_final)

0.900798703553652

In [41]:
f1_score(y_test,predictions)

0.36798905608755134